In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np

from pathlib import Path

from astropy import units as u
from astropy.table import Table 
from astropy.nddata import CCDData
from astropy.coordinates import SkyCoord, EarthLocation
from astropy.time import Time, TimeDeltaSec

import glowing_waffles.differential_photometry as gwdf


## Change the settings below as appropriate for your images

In [ ]:
# This can be a relative or absolute path
# On Windows make this a "raw" string by putting an r before the string, otherwise
# Python interprets the backslashes as escapotential energy sequences.
# Windows example:
# directory_with_images = r'C:\path\to\my\files'
#
# Mac/Linux example:
directory_with_images = '/path/to/my/files'

# If your magnitudes are in the same directory as the images then change to
# directory_with_magnitude_file = directory_with_images
directory_with_magnitude_file = '.'

# This only used to center to make a light curve of your object at the end of this
# notebook.
# If the object you have can't be looked up in simbad this cell will raise an error.
object_of_interest = 'my_object'

if object_of_interest:
    obj_coords = SkyCoord.from_name(object_of_interest)
    
# can be any image from middle of sequence    
sample_image_for_finding_stars = 'sample-image.fit'

input_photometry_file = 'my-file-from-photometry-notebook.fits'

# Set the faintest stars to use in calibrating the magnitudes
# For the new camera (any nights in 2018) use 13
# For the old camera (any older nights) use 16
faintest_mag = 13

# Set the name of the apass column to use for calibration
#r_mag for the new camera
#Rmag for the old
apass_cal_mag = 'r_mag'

# Set the name of the column in our data file for our instrumental magnitude
# Should be mag_inst_r for the new camera, mag_inst_R for the old
feder_instrumental_mag = 'mag_inst_r'

# change this to the name of your input photometry file
output_photometry_file = 'my-input-photometry-file' +'-transformed.fits'

# Set the location of our observatory
feder = EarthLocation("263.54672d", "46.86678d")

In [ ]:
def func(image_directory, sample_image):
    ccd = CCDData.read(Path(image_directory) / sample_image)
    
    apass, apass_x, apass_y, apass_for_transform, in_apass_x, in_apass_y = gwdf.find_apass_stars(ccd)
    vsx, vsx_x, vsx_y, vsx_names = gwdf.find_known_variables(ccd)
    
    apass_all_coords = SkyCoord(apass['RAJ2000'], apass['DEJ2000'], unit='deg')
    apass_for_transform_coords = SkyCoord(apass_for_transform['RAJ2000'], apass_for_transform['DEJ2000'], unit='deg')
    
    return ccd , apass, apass_x, apass_y, apass_for_transform, in_apass_x, in_apass_y, vsx, vsx_x, vsx_y, vsx_names, apass_all_coords, apass_for_transform_coords 

In [ ]:
ccd, apass, apass_x, apass_y, apass_for_transform, in_apass_x, in_apass_y, vsx, vsx_x, vsx_y, vsx_names, apass_all_coords, apass_for_transform_coords = func(directory_with_images, sample_image_for_finding_stars) 

In [ ]:
#ccd = CCDData.read(Path(directory_with_images) / sample_image_for_finding_stars)

In [ ]:
#apass, apass_x, apass_y, apass_for_transform, in_apass_x, in_apass_y = gwdf.find_apass_stars(ccd)
#vsx, vsx_x, vsx_y, vsx_names = gwdf.find_known_variables(ccd)

#apass_all_coords = SkyCoord(apass['RAJ2000'], apass['DEJ2000'], unit='deg')
#apass_for_transform_coords = SkyCoord(apass_for_transform['RAJ2000'], apass_for_transform['DEJ2000'], unit='deg')

## Add column for R filter in case we need it

In [ ]:
apass['Rmag'] = gwdf.filter_transform(apass, 'R', r='r_mag', i='i_mag', transform='ivezic')
apass_for_transform['Rmag'] = gwdf.filter_transform(apass_for_transform, 'R', r='r_mag', i='i_mag', transform='ivezic')

apass['Imag'] = gwdf.filter_transform(apass, 'I', r='r_mag', i='i_mag', transform='ivezic')
apass_for_transform['Imag'] = gwdf.filter_transform(apass_for_transform, 'I', r='r_mag', i='i_mag', transform='ivezic')

## Read in the data file

The `read_data` function also gets the "star_id" for object of interest.

In [ ]:
#read in data file and get star_id for object of interest
def read_data(mag_directory, photometry_input):
    input_mags = Table.read(Path(mag_directory) / photometry_input)
    
    if object_of_interest:
        input_coords = SkyCoord(input_mags['RA'], input_mags['Dec'])
        idx, d2d, d3d = obj_coords.match_to_catalog_sky(input_coords)

        object_of_interest_id = input_mags['star_id'][idx]
        print(object_of_interest_id)
        
    return input_mags, input_coords, idx, d2d, d3d, object_of_interest_id

input_mags, input_coords, idx, d2d, d3d, object_of_interest_id = read_data(directory_with_magnitude_file, input_photometry_file)

In [ ]:
input_mags['mag_cal'] = np.zeros(len(input_mags))
input_mags['zero_point'] = np.zeros(len(input_mags))
input_mags['color_term'] = np.zeros(len(input_mags))
input_mags['t_start_tdb'] = np.zeros(len(input_mags))
input_mags['t_mid_tdb'] = np.zeros(len(input_mags))

mags_grouped = input_mags.group_by('file')

In [ ]:
input_mags.colnames

In [ ]:
print('On image...', end='')
for idx, our_mags in enumerate(mags_grouped.groups):
    print(str(idx) + '..', end='')
    filt = our_mags['filter'][0]
    input_mag_col = 'mag_inst_' + filt
    apass_col = filt + '_mag'
    new_mags, matches, transform = gwdf.transform_magnitudes(our_mags, apass, apass_for_transform, 
                                                faintest_mag_for_transform=faintest_mag,
                                                input_mag_colum=input_mag_col,
                                                catalog_mag_column=apass_col,
                                                catalog_color_column='B-V'
                                               )
    our_mags['mag_cal'][matches] = new_mags.data.filled(np.nan)
    our_mags['mag_cal'][~matches] = np.nan
    our_mags['zero_point'] = transform.parameters[0]
    our_mags['color_term'] = transform.parameters[1]
    t_utc = Time(our_mags['date-obs'], scale='utc', location=feder)
    our_coords = SkyCoord(our_mags['RA'], our_mags['Dec'])
    delta_t_tdb = t_utc.light_travel_time(our_coords)
    t_tdb = t_utc.tdb + delta_t_tdb
    t_tdb_mid = t_tdb + our_mags['exposure'] / 2
    our_mags['t_start_tdb'][:] = t_tdb.jd
    our_mags['t_mid_tdb'][:] = t_tdb_mid.jd

In [ ]:
mags_grouped.sort('t_start_tdb')

In [ ]:
if object_of_interest:
    object_alone = mags_grouped[mags_grouped['star_id'] == object_of_interest_id]
    

## Identify known variables in the field 

### Do the matching and add VSX name column to output table

In [ ]:
def match():
    mag_coords = SkyCoord(mags_grouped['RA'], mags_grouped['Dec'])
    vsx_coords = SkyCoord(vsx['RAJ2000'], vsx['DEJ2000'])
    vsx_idx, d2d, d3d = mag_coords.match_to_catalog_sky(vsx_coords)
    matches = d2d < 3 * u.arcsec
    
    vsx_names = np.array([''] * len(mags_grouped), dtype='S50')
    vsx_names[matches] = vsx['Name'][vsx_idx[matches]]
    mags_grouped['vsx_name'] = vsx_names
    
    return mag_coords, vsx_coords, vsx_idx, d2d, d3d, matches

## Write output to a file

In [ ]:
mags_grouped.write(output_photometry_file)

## Graph transform coefficients


In [ ]:
regrouped = mags_grouped.group_by('file')
agg_regrouped = regrouped.groups.aggregate(np.mean)

In [ ]:
plt.figure(figsize=(15, 5))
plt.plot(agg_regrouped['color_term'], '.')
plt.ylim(-4, 2)
plt.grid()

In [ ]:
plt.figure(figsize=(15, 5))
plt.grid()
plt.plot(agg_regrouped['t_mid_tdb'], agg_regrouped['zero_point'], '.')
#plt.ylim(20, 21)

## Graph of your object, *uncalibrated* magnitudes

In [ ]:
if not object_of_interest:
    raise ValueError('Remaining cells will work only if the object_of_interest is set to a non-empty string.')

In [ ]:
plt.figure(figsize=(15, 5))
plt.plot(object_alone['t_mid_tdb'], object_alone['mag_inst_r'], '.')
plt.ylim(reversed(plt.ylim()))
plt.title('Uncalibrated Magnituds Kelt 1b 12-16-19') #REPLACE THIS WITH MEANINGFUL TITLE
plt.ylabel('instrumental magnitude, r filter')
plt.xlabel('barycentric julian date')
plt.grid()

## Graph of your object, *calibrated* magnitudes

For an exoplanet transit, enter the ingress, midpoint and egress below. Uncomment the cells, fill in your date and time (UTC) and run. In the 3rd cell, uncomment the 2 lines to draw a line at the beginning and end of the transit. 

In [ ]:
#start_time = Time("2018-12-17 04:07:00", scale='utc')
#mid_time = Time("2018-12-17 05:23:00", scale='utc')
#end_time = Time("2018-12-17 06:40:00", scale='utc')

In [ ]:
#print(start_time.tdb.jd)
#print(mid_time.tdb.jd)
#print(end_time.tdb.jd)

In [ ]:
plt.figure(figsize=(15, 5))
plt.plot(object_alone['t_mid_tdb'], object_alone['mag_cal'], '.')
#plt.axvline(x=start_time.tdb.jd)
#plt.axvline(x=end_time.tdb.jd)
plt.ylim(reversed(plt.ylim()))
plt.title('Calibrated Magnituds Kelt 1b 12-16-19') #REPLACE THIS WITH MEANINGFUL TITLE
plt.ylabel('calibrated magnitude, r filter')
plt.xlabel('barycentric julian date')
plt.grid()

Run this cell (only if you have uncommented the above cells) to graph an explanet transit from start to end to see more detail. If you run this cell without uncommenting the lines mentioned above, you will get an error.

In [ ]:
plt.figure(figsize=(15, 5))
plt.plot(object_alone['t_mid_tdb'], object_alone['mag_cal'], '.')
plt.ylim(reversed(plt.ylim()))
plt.xlim(start_time.tdb.jd, end_time.tdb.jd)
plt.title('in-detail transit')
plt.ylabel('calibrated magnitude, r filter')
plt.xlabel('barycentric julian date')
plt.grid()